In [2]:
import numpy as np

# Load the saved features and labels
features = np.load('./train_mfcc_features/features.npy')
labels = np.load('./train_mfcc_features/labels.npy')

# Check the shapes to confirm successful loading
print("features shape:", features.shape)
print("labels shape:", labels.shape)


features shape: (5334, 184)
labels shape: (5334,)


In [3]:
from sklearn.model_selection import train_test_split

# Split into train and test sets with a fixed random_state for reproducibility
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size=0.2, random_state=42  # Adjust `test_size` as needed
)

# Confirm shapes of the splits
print("Training features shape:", train_features.shape)
print("Training labels shape:", train_labels.shape)
print("Test features shape:", test_features.shape)
print("Test labels shape:", test_labels.shape)

Training features shape: (4267, 184)
Training labels shape: (4267,)
Test features shape: (1067, 184)
Test labels shape: (1067,)


In [4]:
from sklearn.model_selection import train_test_split

# Split into train and test sets with a fixed random_state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42  # Adjust `test_size` as needed
)

# Confirm shapes of the splits
print("Training features shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Test features shape:",  X_test.shape)
print("Test labels shape:", y_test.shape)

Training features shape: (4267, 184)
Training labels shape: (4267,)
Test features shape: (1067, 184)
Test labels shape: (1067,)


In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define parameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}


In [6]:
# Initialize KNN classifier
knn = KNeighborsClassifier()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

# Run grid search on training data
# grid_search.fit(train_features, train_labels)
grid_search.fit(X_train, y_train)



Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ..metric=euclidean, n_neighbors=3, weights=distance; total time=   0.1s
[CV] END ..metric=euclidean, n_neighbors=3, weights=distance; total time=   0.1s
[CV] END ...metric=euclidean, n_neighbors=3, weights=uniform; total time=   0.1s
[CV] END ...metric=euclidean, n_neighbors=3, weights=uniform; total time=   0.1s
[CV] END ...metric=euclidean, n_neighbors=3, weights=uniform; total time=   0.1s
[CV] END ...metric=euclidean, n_neighbors=3, weights=uniform; total time=   0.1s
[CV] END ...metric=euclidean, n_neighbors=3, weights=uniform; total time=   0.1s
[CV] END ..metric=euclidean, n_neighbors=3, weights=distance; total time=   0.2s
[CV] END ..metric=euclidean, n_neighbors=3, weights=distance; total time=   0.0s
[CV] END ...metric=euclidean, n_neighbors=5, weights=uniform; total time=   0.0s
[CV] END ..metric=euclidean, n_neighbors=3, weights=distance; total time=   0.0s
[CV] END ...metric=euclidean, n_neighbors=5, we

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 7, 9, 11],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy', verbose=2)

In [7]:
# Best model and parameters
best_knn = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best Parameters:", best_params)


Best Parameters: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}


In [8]:
from sklearn.metrics import accuracy_score, classification_report

# Use the best model to predict on the validation set
test_predictions = best_knn.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, test_predictions)
print(f"\nTesting Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:\n", classification_report(y_test, test_predictions))


Testing Accuracy: 99.81%

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       129
           1       1.00      1.00      1.00       152
           2       1.00      1.00      1.00       153
           3       1.00      1.00      1.00       126
           4       1.00      0.98      0.99       127
           5       1.00      1.00      1.00       140
           6       0.98      1.00      0.99       102
           7       1.00      1.00      1.00       138

    accuracy                           1.00      1067
   macro avg       1.00      1.00      1.00      1067
weighted avg       1.00      1.00      1.00      1067



# Creating csv file to be uploaded to zindi

In [9]:
val_features = np.load('./val_mfcc_features/features.npy')
val_features.shape

(2946, 196)

In [10]:
target_length = 184  # Target mfcc length used from train features

if val_features.shape[1] > target_length:
    # Truncate to the target length
    val_features = val_features[:, :target_length]
elif val_features.shape[1] < target_length:
    # Pad with zeros to the target length
    val_features = np.pad(val_features, ((0, 0), (0, target_length - val_features.shape[1])), 'constant')


In [11]:
val_features.shape

(2946, 184)

In [12]:
import pandas as pd
val_metadata = pd.read_csv('./Test_1.csv')
val_metadata

,id,audio_filepath,duration
0,id_u5iqtgjzhx,id_u5iqtgjzhx.wav,2.35800
1,id_l7ebzcfk5e,id_l7ebzcfk5e.wav,3.22100
2,id_jbzci8uepl,id_jbzci8uepl.wav,2.94898
3,id_jzil0fw5vs,id_jzil0fw5vs.wav,4.33100
4,id_o7mrvf5wj7,id_o7mrvf5wj7.wav,1.85600
...,...,...,...
2941,id_gx8hrglpc9,id_gx8hrglpc9.wav,1.33800
2942,id_9bxbutuh8z,id_9bxbutuh8z.wav,1.98400
2943,id_d0tudlf5gn,id_d0tudlf5gn.wav,1.33800
2944,id_xuwlsfztrd,id_xuwlsfztrd.wav,2.17800


In [13]:
predicted_classes = best_knn.predict(val_features)

In [14]:
output_df = pd.DataFrame({
    'id': val_metadata['id'],
    'class': predicted_classes
})
output_df

,id,class
0,id_u5iqtgjzhx,4
1,id_l7ebzcfk5e,0
2,id_jbzci8uepl,7
3,id_jzil0fw5vs,0
4,id_o7mrvf5wj7,4
...,...,...
2941,id_gx8hrglpc9,0
2942,id_9bxbutuh8z,0
2943,id_d0tudlf5gn,0
2944,id_xuwlsfztrd,3


In [15]:
output_df.to_csv('predictions.csv', index=False)

In [16]:
# Get class mappings:
md = pd.read_csv('./Train.csv')
unique_classes = md['class'].unique()
class_mapping = {class_name: idx for idx, class_name in enumerate(unique_classes)}
class_mapping

{'left': 0,
 'down': 1,
 'go': 2,
 'right': 3,
 'up': 4,
 'yes': 5,
 'no': 6,
 'stop': 7}

In [17]:
inverse_class_mapping = {v: k for k, v in class_mapping.items()}
inverse_class_mapping

{0: 'left',
 1: 'down',
 2: 'go',
 3: 'right',
 4: 'up',
 5: 'yes',
 6: 'no',
 7: 'stop'}

In [18]:
output_df['class'] = output_df['class'].map(inverse_class_mapping)
output_df

,id,class
0,id_u5iqtgjzhx,up
1,id_l7ebzcfk5e,left
2,id_jbzci8uepl,stop
3,id_jzil0fw5vs,left
4,id_o7mrvf5wj7,up
...,...,...
2941,id_gx8hrglpc9,left
2942,id_9bxbutuh8z,left
2943,id_d0tudlf5gn,left
2944,id_xuwlsfztrd,right


In [19]:
output_df.to_csv('predictions.csv', index=False)